# Week 1 - Homework
## Question 1

In [110]:
!curl http://localhost:9200

157321.54s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


{
  "name" : "56ad20e5099f",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "OEHFVm2KSOi2fkjtip28-Q",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Question 2

In [84]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [85]:
from elasticsearch import Elasticsearch
es = Elasticsearch('http://localhost:9200')
es.info()

ObjectApiResponse({'name': '65ad251f2afd', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'gJTfKtocQXuXjpgE_YvfSQ', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [90]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course_question'

if not es.indices.exists(index = index_name):
    es.indices.create(index = index_name, body = index_settings)
    for doc in documents:
        es.index(index = index_name, document=doc)

## Question 3

In [91]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
                }
            }
        }
    
    response = es.search(index = index_name, body = search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        
        
    return response

In [92]:
query = "How do execute a command on a Kubernetes pod?"

In [93]:
for hit in elastic_search(query)['hits']['hits']:
    print(hit['_score'])

44.50556
35.433445
33.70974
33.2635
32.589073


## Question 4

In [94]:
query = "How do copy a file to a Docker container?"

In [95]:
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": [
                    {"term": {
                        "course": "machine-learning-zoomcamp"
                    }}
                ]
                }
            }
        }
    
response = es.search(index = index_name, body = search_query)
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
    
for rd in result_docs:
    print(rd['question'])

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


## Question 5

In [97]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [98]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [99]:
prompt = prompt_template.format(
    question = query,
    context= "\n\n".join([
        context_template.format(
            question = rd.get("question"),
            text = rd.get("text")
        ) for rd in result_docs
    ])
)

In [103]:
len(prompt)

1446

## Question 6

In [105]:
import tiktoken

In [106]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [108]:
len(encoding.encode(prompt))

320